# Traitement de données, 2022, TD2

Ceci est un notebook python, qui va être utilisé pour le TD d'ajourd'hui. Il se trouve sur Google Colab, pour être facile à partager. Attention : Vous devriez pouvoir l'éditer, mais tant que vous ne l'avez pas téléchargé ou copié, aucune de vos modifications ne seront sauvées.

Pour ajouter vos solutions aux questions vous pouvez soit

Créer une copie de ce notebook sur votre propre compte Google (Fichier>Sauvegarder un copie sur Drive).
Télécharger le notebook (Fichier>Télécharger>.ipynb) et l'éditer/executer sur jupyter lab/notebook.
Télécharger le script python correspondant (Fichier>Télécharger>.py), l'éditer avec un éditeur de texte/code local et l'executer avec l'interpreteur local (python(3)), possiblement en utilisant un environement virtuel conda (ou tout faire dans un IDE).

## I- Données de MusicBrainz

MusicBrainz est une vaste base de données musical en ligne. Elle contient des informations sur les musiciens, les albums, les chansons, leur localisation, etc. L'ensemble des données de MusicBrainz est disponible au travers d'une [API](https://musicbrainz.org/doc/MusicBrainz_API).

On s'intéresse ici aux musiciens de Nancy. Pour récupérer des informations sur les relations entre types d'objets, on va utiliser la fonction "browse" de l'API de MusicBrainz.

Comme expliqué dans la documentationd de l'API, sachant que l'URL de base de l'API est https://musicbrainz.org/ws/2/, une requete de type browse prend la forme 
```
 /<RESULT_ENTITY_TYPE>?<BROWSING_ENTITY_TYPE>=<MBID>&limit=<LIMIT>&offset=<OFFSET>&inc=<INC>
```

Ici, on s'interesse aux artistes localisés à Nancy, donc :
* RESULT_ENTYT_TYPE est `artist`
* BROWSING_ENTITY_TYPE est `area`
* MBID est l'identifiant de Nancy dans MusicBrainz, i.e. `2b7c4074-dded-4703-a7de-e1d045c34b5c`

On voudra aussi que l'informatin sur le genre de musique, si disponible, soit inclu dans le résultat. Donc on mettra la valeur `genres` pour INC.

On veut obtenir un maximum de résultats, donc on mettra `100` pour LIMIT, qui est le nombre maximum de résultats que MusicBrainz peut renvoyer. 

On laissera OFFSET à `0` pour le moment. 

### Simple requete

Commencez par utiliser la librairie `requests` de python pour appeller l'API de MusicBrainz avec ces paramètres et afficher le resultat : 
``` 
import requests
r = requests.get(...)
print(r.text)
```

### Résultat en JSON
Vous remarquerez que MusicBrainz, par default, renvoie du XML. Pour lui demander de renvoyer du JSON à la place, il faut utiliser l'entête `Accept` dans la requête HTTP realisée par `requests` et lui donner la valeur `application/json`. Pour cela, utlisez l'argument `headers` de la function `get` avec comme valeur le dictionnaire :
```
{"Accept": "application/json"}
```
Le resultat de la requête (`r.text`) contient maintenant du JSON. 

### Créer la DataFrame

La fonction `r.json()` de la requête créée renvoit un objet (dictionnaire) python créé à partir du résultat de la requête en JSON. L'attribut `"artists"` de ce dictionnaire contient (normalement) la liste des artistes connus de MusicBrainz qui sont localisés à Nancy. Utilisez cette liste pour créer une DataFrame pandas (`pd.DataFrame(...)`) et affichez la. 

In [ ]:
import requests 
import json
header = {"Accept": "application/json"}
r = requests.get("https://musicbrainz.org/ws/2/artist?area=2b7c4074-dded-4703-a7de-e1d045c34b5c&limit=100&offset=0&inc=genres", headers=header)
json.loads(r.text)
data = r.json()



In [ ]:
import pandas as pd
mdf1 = pd.DataFrame(data["artists"])
mdf1.shape

(100, 18)

### Completer les données

MusicBrainz contient des informations sur ~195 musiciens/groupes de Nancy. La requête précédente n'en contient que 100. Changez la valeur du paramêtre `OFFSET` dans une nouvelle requête pour créer une nouvelle DataFrame contenant les informations pour tout artiste de Nancy à un rang entre 100 et 200. 

Concatennez verticalement la première DataFrame et la deuxième pour créer une troisième DataFrame.  

In [ ]:
header = {"Accept": "application/json"}
r = requests.get("https://musicbrainz.org/ws/2/artist?area=2b7c4074-dded-4703-a7de-e1d045c34b5c&limit=100&offset=100&inc=genres", headers=header)
json.loads(r.text)
data2 = r.json()

In [ ]:
mdf2 = pd.DataFrame(data2["artists"])
mdf2

mdf = pd.concat([mdf1, mdf2], axis=0)

mdf

,name,end_area,sort-name,begin_area,ipis,country,gender,disambiguation,id,genres,type,type-id,area,begin-area,life-span,end-area,isnis,gender-id
0,Les Boum Bomo's,None,"Boum Bomo's, Les",None,[],None,None,French parodic punk band,01b40f9c-46fc-48ac-8c5c-24c657ea2b3c,[],Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'type': None, 'name': 'Nancy', 'id': '2b7c407...",None,"{'ended': False, 'end': None, 'begin': None}",None,[],None
1,Romain Aweduti,None,"Aweduti, Romain",None,[],None,None,,047eab94-0717-430d-9209-c21e443dcc50,[],Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",None,"{'begin': None, 'end': None, 'ended': False}",None,[],None
2,1st Round,None,1st Round,None,[],None,None,,0588e91a-4d5b-40d9-98e5-bae085aae7dd,[],Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'disambiguation': '', 'type-id': None, 'sort-...",None,"{'ended': False, 'end': None, 'begin': None}",None,[],None
3,Fractal Universe,None,Fractal Universe,"{'name': 'Nancy', 'id': '2b7c4074-dded-4703-a7...",[],None,None,,0ac72b79-5511-447f-bd5e-e5caa92974f9,"[{'count': 1, 'disambiguation': '', 'name': 'd...",Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'name': 'Nancy', 'id': '2b7c4074-dded-4703-a7...","{'name': 'Nancy', 'id': '2b7c4074-dded-4703-a7...","{'begin': '2013', 'end': None, 'ended': False}",None,[0000000469061653],None
4,Fuzzphorus,None,Fuzzphorus,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",[],None,None,,0ce34c84-0190-4d02-bbfd-467fc936df52,"[{'count': 1, 'disambiguation': '', 'name': 'd...",Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'name': 'Verdun', 'id': 'a3adf012-261e-4733-b...","{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...","{'begin': '2011', 'end': None, 'ended': False}",None,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Henri Ledroit,"{'type-id': None, 'sort-name': 'Nancy', 'type'...","Ledroit, Henri",None,[],FR,Male,countertenor,f1b54b91-ab2b-4614-9b1b-3be81bab1d19,[],Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'type-id': None, 'id': '08310658-51eb-3801-80...",None,"{'ended': True, 'begin': '1946-03-11', 'end': ...","{'type-id': None, 'sort-name': 'Nancy', 'type'...",[000000006309157X],36d3d30a-839d-3eda-8cb3-29be4384e4a9
92,Charles Keller,"{'type-id': None, 'sort-name': 'Nancy', 'disam...","Keller, Charles","{'name': 'Mulhouse', 'type': None, 'disambigua...",[],FR,Male,dit Jacques Turbin,f3e83a04-bb8c-416b-b064-c0faf73f4a36,[],Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'type': None, 'name': 'France', 'disambiguati...","{'name': 'Mulhouse', 'type': None, 'disambigua...","{'end': '1913-07-19', 'ended': True, 'begin': ...","{'type-id': None, 'sort-name': 'Nancy', 'disam...",[0000000395565612],36d3d30a-839d-3eda-8cb3-29be4384e4a9
93,Rodger N'Furter,None,"N'Furter, Rodger",None,[],None,Male,,f68e41df-199c-47fa-b609-22a8c721c0de,[],Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",None,"{'begin': None, 'ended': False, 'end': None}",None,[],36d3d30a-839d-3eda-8cb3-29be4384e4a9
94,Karne,None,Karne,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",[],FR,None,French black metal,f9075ca2-2eaa-4b4f-8c21-4e6b88ff429e,"[{'count': 2, 'id': '12568698-97cc-49a9-80ab-9...",Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'type-id': None, 'id': '08310658-51eb-3801-80...","{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...","{'begin': '2011', 'ended': False, 'end': None}",None,[],None


### Extraire un seul genre

Vous remarquerez que la colonne `genres` contient des valeurs de type liste, car chaque artiste peut avoir plusieurs genres. Créez une nouvelles colonne `genre` (sans 's') qui ne contient que le premier genre pour chaque artiste de la liste de la colone `genres` (si elle existe).

Utilisez les fonctions `unique()` et `value_counts()` pour verifier quels genres existent et conbien il y a d'artistes de chaque.

In [ ]:
mdf["genre"] = mdf.genres.apply(lambda x : x[0]["name"] if len(x)>0 and "name" in x[0] else None)
mdf.drop("genres", axis=1, inplace=True)
mdf

,name,end_area,sort-name,begin_area,ipis,country,gender,disambiguation,id,type,type-id,area,begin-area,life-span,end-area,isnis,gender-id,genre
0,Les Boum Bomo's,None,"Boum Bomo's, Les",None,[],None,None,French parodic punk band,01b40f9c-46fc-48ac-8c5c-24c657ea2b3c,Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'type': None, 'name': 'Nancy', 'id': '2b7c407...",None,"{'ended': False, 'end': None, 'begin': None}",None,[],None,None
1,Romain Aweduti,None,"Aweduti, Romain",None,[],None,None,,047eab94-0717-430d-9209-c21e443dcc50,Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",None,"{'begin': None, 'end': None, 'ended': False}",None,[],None,None
2,1st Round,None,1st Round,None,[],None,None,,0588e91a-4d5b-40d9-98e5-bae085aae7dd,Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'disambiguation': '', 'type-id': None, 'sort-...",None,"{'ended': False, 'end': None, 'begin': None}",None,[],None,None
3,Fractal Universe,None,Fractal Universe,"{'name': 'Nancy', 'id': '2b7c4074-dded-4703-a7...",[],None,None,,0ac72b79-5511-447f-bd5e-e5caa92974f9,Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'name': 'Nancy', 'id': '2b7c4074-dded-4703-a7...","{'name': 'Nancy', 'id': '2b7c4074-dded-4703-a7...","{'begin': '2013', 'end': None, 'ended': False}",None,[0000000469061653],None,death metal
4,Fuzzphorus,None,Fuzzphorus,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",[],None,None,,0ce34c84-0190-4d02-bbfd-467fc936df52,Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'name': 'Verdun', 'id': 'a3adf012-261e-4733-b...","{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...","{'begin': '2011', 'end': None, 'ended': False}",None,[],None,desert rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Henri Ledroit,"{'type-id': None, 'sort-name': 'Nancy', 'type'...","Ledroit, Henri",None,[],FR,Male,countertenor,f1b54b91-ab2b-4614-9b1b-3be81bab1d19,Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'type-id': None, 'id': '08310658-51eb-3801-80...",None,"{'ended': True, 'begin': '1946-03-11', 'end': ...","{'type-id': None, 'sort-name': 'Nancy', 'type'...",[000000006309157X],36d3d30a-839d-3eda-8cb3-29be4384e4a9,None
92,Charles Keller,"{'type-id': None, 'sort-name': 'Nancy', 'disam...","Keller, Charles","{'name': 'Mulhouse', 'type': None, 'disambigua...",[],FR,Male,dit Jacques Turbin,f3e83a04-bb8c-416b-b064-c0faf73f4a36,Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'type': None, 'name': 'France', 'disambiguati...","{'name': 'Mulhouse', 'type': None, 'disambigua...","{'end': '1913-07-19', 'ended': True, 'begin': ...","{'type-id': None, 'sort-name': 'Nancy', 'disam...",[0000000395565612],36d3d30a-839d-3eda-8cb3-29be4384e4a9,None
93,Rodger N'Furter,None,"N'Furter, Rodger",None,[],None,Male,,f68e41df-199c-47fa-b609-22a8c721c0de,Person,b6e035f4-3ce9-331c-97df-83397230b0df,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",None,"{'begin': None, 'ended': False, 'end': None}",None,[],36d3d30a-839d-3eda-8cb3-29be4384e4a9,None
94,Karne,None,Karne,"{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...",[],FR,None,French black metal,f9075ca2-2eaa-4b4f-8c21-4e6b88ff429e,Group,e431f5f6-b5d2-343d-8b36-72607fffb74b,"{'type-id': None, 'id': '08310658-51eb-3801-80...","{'id': '2b7c4074-dded-4703-a7de-e1d045c34b5c',...","{'begin': '2011', 'ended': False, 'end': None}",None,[],None,black metal


### Extraire la date de début
Vous remarquerez aussi que la colonne `life-span` contient un dictionnaire avec la date de début et la date de fin de carrière des artistes pour lesquelles ces informations sont disponibles. 

Créez une nouvelle colonne `start-date` contenant **l'année** de début de carrière de chaque artiste pour lequel elle est connue. 

Quelle est l'intervalle d'année dans cette colonne ? L'année moyenne de début de carrière ? Dans quelle période la majorité des artistes ont commencés leur carrière ? 

In [ ]:

mdf["start-date"] = mdf["life-span"].apply(lambda x : x["begin"][0:4] if x["begin"] else None).apply(lambda x : int(x) if x and x.isnumeric() else None)

mdf["start-date"].describe()

count     128.000000
mean     1946.351562
std        87.760464
min      1433.000000
25%      1930.500000
50%      1974.500000
75%      1997.250000
max      2020.000000
Name: start-date, dtype: float64

### Réduire la DataFrame et index

Réduisez la DataFrame au colonnes `sort-name`, `name`, `gender`, `genre`, `disambiguation`, `country`, `start-date` et faites de `sort-name` son index. Sauvez le résultat dans un fichier csv.

In [ ]:
musicbrainz = mdf[["sort-name","name", "gender", "genre", "country", "start-date"]].set_index("sort-name")
musicbrainz.to_csv("musicbrainz.csv")


## II- Données de Wikidata

On va ici utiliser le point d'accès SPARQL de [wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page). Tout d'abord assurez vous d'avoir la librairie `SPARQLWrapper` installée dans votre environment. Dans Google Colab, cela peut être réalisé avec la commande 
```
!pip install SPARQLWrapper
```

In [ ]:
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


La requête SPARQL ci-dessous obtient des informations sur le nom de l'artiste (`artLabel`), son sexe (`genderLabel`), la date de début de carrière (`startdate`), l'instrument joué (`instLabel`), le genre (`genreLabel`) et le "label" qui le/la représente (`labelLabel`) pour des musiciens ou des groupes ayant une relation avec la ville de Nancy. 

In [ ]:
wdquery="""
SELECT distinct ?art ?artLabel ?genderLabel ?startdate ?instLabel ?genreLabel ?labelLabel
WHERE 
{
  {?art wdt:P106 ?occ.
  ?occ wdt:P31 wd:Q66715801.
  ?art ?p wd:Q40898.
  optional{?art wdt:P21 ?gender.}
  optional {?art wdt:P1303 ?inst}.
  optional {?art wdt:P136 ?genre}.
  optional {?art wdt:P264 ?label}.
  optional {?art wdt:P2031 ?startdate}.
   } union { 
  ?art wdt:P31 wd:Q215380.
  ?art ?p wd:Q40898.
  optional {?art wdt:P21 ?gender.}
  optional {?art wdt:P1303 ?inst}.
  optional {?art wdt:P136 ?genre}.
  optional {?art wdt:P264 ?label}.
  optional {?art wdt:P2031 ?startdate}.
    }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""

Sachant que l'URL du point d'accès SPARQL de wikidata est 
```
https://query.wikidata.org/sparql
```
Créez une DataFrame avec les résulats de cette requête et incluant les colonnes `name`, `gender`, `start`, `genre`, `instrument`, et `label`.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url="https://query.wikidata.org/sparql"
sparql=SPARQLWrapper(endpoint_url)
sparql.setQuery(wdquery)
sparql.setReturnFormat(JSON)
result = sparql.query().convert()
result

{'head': {'vars': ['art',
   'artLabel',
   'genderLabel',
   'startdate',
   'instLabel',
   'genreLabel',
   'labelLabel']},
 'results': {'bindings': [{'art': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q696034'},
    'artLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Jean-Paul-Égide Martini'},
    'genderLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'male'},
    'genreLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'opera'}},
   {'art': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q13537120'},
    'artLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Johann Vexo'},
    'genderLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'male'},
    'instLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'pipe organ'}},
   {'art': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q13537120'},
    'artLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Johann Vexo'},
    'genderLabel': {'xml:lang

In [ ]:
import requests

header={"Accept": "application/json"}
r=requests.get(endpoint_url+"?query="+wdquery, headers=header)
# r.text
data = r.json()


In [ ]:
name = [x["artLabel"]["value"] for x in data["results"]["bindings"]]
gender = [x["genderLabel"]["value"] if "genderLabel" in x else None for x in data["results"]["bindings"]]
sd = [x["startdate"]["value"] if "startdate" in x else None for x in data["results"]["bindings"]]
genre = [x["genreLabel"]["value"] if "genreLabel" in x else None for x in data["results"]["bindings"]]
inst = [x["instLabel"]["value"] if "instLabel" in x else None for x in data["results"]["bindings"]]
label = [x["labelLabel"]["value"] if "labelLabel" in x else None for x in data["results"]["bindings"]]

wdf = pd.DataFrame({"name": name, "gender":gender, "startdate": sd, "genre": genre, "instrument": inst, "label": label})
wdf

,name,gender,startdate,genre,instrument,label
0,Jean-Paul-Égide Martini,male,None,opera,None,None
1,Johann Vexo,male,None,None,pipe organ,None
2,Johann Vexo,male,None,None,organ,None
3,Romain Descharmes,male,None,None,piano,None
4,Charlélie Couture,male,None,blues rock,harmonica,None
...,...,...,...,...,...,...
76,Jean Parisot,male,None,None,None,None
77,Oskar Matzerath,male,None,jazz,tin drum,None
78,Annick Chartreux,female,None,None,None,None
79,Bruno Peltre,male,None,None,piano,None


### Année de début

La colonne `start` contient des dates entières. Créez une colone `start-date` ne contenant que l'année. Calculez les statistiques descriptives pour répondre aux mêmes questions que pour les données de MusicBrainz sur l'année de début. 

In [ ]:
from datetime import datetime

wdf["start-date"] = wdf.startdate.apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").year if x else None)
wdf.drop("startdate", axis=1, inplace=True)
wdf


,name,gender,genre,instrument,label,start-date
0,Jean-Paul-Égide Martini,male,opera,None,None,NaN
1,Johann Vexo,male,None,pipe organ,None,NaN
2,Johann Vexo,male,None,organ,None,NaN
3,Romain Descharmes,male,None,piano,None,NaN
4,Charlélie Couture,male,blues rock,harmonica,None,NaN
...,...,...,...,...,...,...
76,Jean Parisot,male,None,None,None,NaN
77,Oskar Matzerath,male,jazz,tin drum,None,NaN
78,Annick Chartreux,female,None,None,None,NaN
79,Bruno Peltre,male,None,piano,None,NaN


### Réduire la DataFrame 
Réduisez la DataFrame aux colonnes `name`, `gender`, `start-date`, `genre`, `instrument`, `label` et sauvez le résultat dans un fichier csv.

In [ ]:
wikidata = wdf[["name", "gender", "start-date", "genre", "instrument", "label"]]
wikidata.to_csv("wikidata.csv")


## III- Intégration

Arrivé à ce point, vous devriez avoir deux DataFrames, une pour MusicBrainz et une pour wikidata, correspondant aux fishiers CSV [musicbrainz.csv](https://mdaquin.github.io/t/TdD_TD1_2021/musicbrainz.csv) et [wikidata.csv](https://mdaquin.github.io/t/TdD_TD1_2021/wikidata.csv).

Créez une nouvelle DataFrame par jointure interne de ce deux DataFrame sur la colonne `name`.

Si vous n'avez pas réussi les exercices précédents, re-créez d'abord les DataFrames en utlisant les fichiers CSV.


In [ ]:
fdf = pd.merge(musicbrainz, wikidata, how="inner", on="name")
fdf[fdf.gender_x.apply(lambda x : x.lower() if x else None) != fdf.gender_y]

,name,gender_x,genre_x,country,start-date_x,gender_y,start-date_y,genre_y,instrument,label
28,KaS Product,None,coldwave,None,1980.0,None,1980.0,new wave,None,"RCA Records, Inc."


Vous remarquerez que les colonnes communes (à part `name`) dans le deux DataFrames ont été rennomées. Par exemple la colonne `gender` de MusicBrainz est devenue `gender_x` (si la DataFrame pour MusicBrainz était la première dans la jointure) et celle de la DataFrame de wikidata est devenue `gender_y`. 

Re-Créez une nouvelle colonne `gender` qui prend les valeurs de sexe provenant de MusicBrainz. Quelle est la propotions d'homme et de femmes dans cette DataFrame ?

In [ ]:
fdf.rename(columns={"gender_x": "gender"}, inplace=True)
fdf.drop("gender_y", axis=1, inplace=True)
fdf

,name,gender,genre_x,country,start-date_x,start-date_y,genre_y,instrument,label
0,Najoua Belyzel,Female,None,FR,1981.0,2005.0,electro,voice,Scorpio Music
1,Guy Fallot,Male,None,FR,1927.0,NaN,None,cello,None
2,Charles Dalmorès,Male,None,FR,1871.0,NaN,None,voice,None
3,Marie-Hélène Bernard,Female,None,FR,1953.0,NaN,None,None,None
4,Charles Baudiot,Male,None,FR,1773.0,NaN,None,cello,None
5,Charlélie Couture,Male,None,FR,1956.0,NaN,blues rock,harmonica,None
6,Charlélie Couture,Male,None,FR,1956.0,NaN,blues rock,guitar,None
7,Charlélie Couture,Male,None,FR,1956.0,NaN,French rock,voice,None
8,Charlélie Couture,Male,None,FR,1956.0,NaN,French rock,piano,None
9,Charlélie Couture,Male,None,FR,1956.0,NaN,French rock,harmonica,None


De même les colonnes `genre` ont été renommées en `genre_x` et `genre_y`.

Calculez les proportions de rangées dans la DataFrame de jointure pour lesquelles les deux colonnes "genres" ont une valeur, par rapport à toutes celles pour lesquelles la colonne `gender_x` a une valeur et à toutes celles pour lesquelles la colonne `gender_y` a une valeur.

La fonction `.notnull()` peut être utilisée pour vérifier, dans un filtre pandas, si une colonne a une valeur (qui n'est pas `None` ou `NaN`). De même la fonction `.isnull()` peut être utilisée pour selectionner les valeurs `None` dans un filtre. 

Re-créez une nouvelle colonne `genre` qui prend sa valeur dans `genre_y` si elle existe, et sinon dans `genre_x`. 

In [ ]:
print("genres dans musicbrainz", fdf.genre_x.count())
print("genres dans wikidata", fdf.genre_y.count())
print("genres dans les deux", len(fdf[fdf.genre_x.notnull() & fdf.genre_y.notnull()]))

def getGenre(l):
  if l.genre_y: return l.genre_y
  return l.genre_x

fdf["genre"] = fdf.apply(getGenre, axis=1)
fdf.drop(["genre_x", "genre_y"], axis=1, inplace=True)
fdf

genres dans musicbrainz 7
genres dans wikidata 21
genres dans les deux 5


,name,gender,country,start-date_x,start-date_y,instrument,label,genre
0,Najoua Belyzel,Female,FR,1981.0,2005.0,voice,Scorpio Music,electro
1,Guy Fallot,Male,FR,1927.0,NaN,cello,None,None
2,Charles Dalmorès,Male,FR,1871.0,NaN,voice,None,None
3,Marie-Hélène Bernard,Female,FR,1953.0,NaN,None,None,None
4,Charles Baudiot,Male,FR,1773.0,NaN,cello,None,None
5,Charlélie Couture,Male,FR,1956.0,NaN,harmonica,None,blues rock
6,Charlélie Couture,Male,FR,1956.0,NaN,guitar,None,blues rock
7,Charlélie Couture,Male,FR,1956.0,NaN,voice,None,French rock
8,Charlélie Couture,Male,FR,1956.0,NaN,piano,None,French rock
9,Charlélie Couture,Male,FR,1956.0,NaN,harmonica,None,French rock


Re-créez un colonne `start-date` prennant ses valeurs dans `start-date_x` et réduisez la DataFrame aux colonnes `name`, `gender`, `genre`, `country`, `start-date`, `instrument` et `label`.

Sauvez la DataFrame dans un ficher CSV.

# nettoyage

La dataframe ci-dessus n'est pas super propre. Par exemple, le même artiste peut apparaitre plusieurs fois. Combien de fois cela arrive t'il ? Quelle est la proportion d'artiste dont le nom est dupliqué dans la dataframe ?

Créez une nouvelle DataFrame (une nouvelle variable) qui ne conserve qu'un seul des artites dont le nom est dupliqué et mettre le nom de l'artiste en index de cette nouvelle dataframe. 

Quelle est la colonne avec le plus de valeurs manquantes (`None` ou `NaN`) ?

Remplacer dans la colonne `genre` toutes les valeurs manquantes par la chaine de caractère "not specified".


Il y a t'il des valeurs aberrantes dans la colonne `start-date` ? Et si on considère les valeurs à plus de deux écarts type de la moyenne ? 

Remplacez ces valeurs par `NaN` (ce ne sont pas vraiment des valeurs abérrantes, mais on fait comme si).